# Detecção Facial

## Treinamento do Modelo

### Preparação do Dataset

Os dados de treinamento para o YOLO precisam estar em um modelo especifico para o funcionamento. Portanto é necessário separar os dados em uma pasta de treino e validação, em que ambos devem estar uma uma pasta de imagens, além disso é necessário ter uma pasta de 'labels' com as mesmas pastas de treino e validação com as informações de classes e 'caixas' a serem treinadas pelo modelo.

Modelo diretório:

data/

----images/

--------train/

--------val/

----labels/

--------train/
        
--------val/    

In [ ]:
import pandas as pd

# Criação do arquivo de labels

# Criação dos parâmetros de X e Y centrais, além da altura e comprimento da caixa
df = pd.read_csv('faces.csv')
df['Xc'] = (df['x1'] + df['x0'])/2
df['Yc'] = (df['y1'] + df['y0'])/2
df['Box_width'] = df['x1'] - df['x0']
df['Box_height'] = df['y1'] - df['y0']

# Parâmenros de X e Y centrais, além da altura e comprimento da caixa
#  como proporção das dimensões da imagem
df_final = pd.DataFrame()
df_final['img'] = df['image_name']
df_final['class'] = 0
df_final['Xc'] = df['Xc']/df['width']
df_final['Yc'] = df['Yc']/df['height']
df_final['height'] = df['Box_height']/df['height']
df_final['width'] = df['Box_width']/df['width']


# Separação dos dados de treino
train = df_final.loc[df_final['img'].apply(lambda x: int(x[:-4])) > 480]

for i, row in train.iterrows():
    clas = round(row['class'], 4)
    xc = round(row['Xc'], 4)
    yc =round(row['Yc'], 4)
    height = round(row['height'], 4)
    width = round(row['width'], 4)

    f = open(f"train/{row['img'][:-4]}.txt", "a")
    f.write(f'{clas} {xc} {yc} {height} {width}\n')
    f.close()

# Separação dos dados de validação
val = df_final.loc[df_final['img'].apply(lambda x: int(x[:-4])) <= 480]

for i, row in val.iterrows():
    clas = round(row['class'], 4)
    xc = round(row['Xc'], 4)
    yc =round(row['Yc'], 4)
    height = round(row['height'], 4)
    width = round(row['width'], 4)

    f = open(f"val/{row['img'][:-4]}.txt", "a")
    f.write(f'{clas} {xc} {yc} {height} {width}\n')
    f.close()

Além disso, é necessário criar um arquivo .yaml com as informações sobre o dataset, os labels, e as classes. 

O seguinte modelo foi usado:

path: "./data"

train: images/train

val: images/val

nc: 1

names: ['Face']

### Treinamento

In [43]:
import torch
torch.cuda.is_available()

True

O modelo será treinado diversas vezes para que tenha uma boa precisão na construção da 'caixa'.

O melhor modelo gerado ficará no caminho 'runs\detect\FacialDetection\weights\best.pt'

In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')

# Train the model
model.train(data='face_detection.yaml', name='FacialDetection', seed=42, epochs=100, patience=40, rect=True, optimizer='Adam')
valid_results = model.val()
print(valid_results)

# Função para detecção e reconhecimento facial

In [41]:
import cv2
from ultralytics import YOLO
from deepface import DeepFace
import numpy as np
import warnings
warnings.filterwarnings("ignore")

model = YOLO('best.pt')

def process_image(image):
    # Detecção Facial
    results = model.predict(image)
    # Posições dos rostos
    box = results[0].boxes.xyxy.tolist()

    for face in box:
        try:
            # Pontos XY
            x0 = int(face[0])
            y0 = int(face[1])
            x1 = int(face[2])
            y1 = int(face[3])

            # Recorte do rosto
            individual_face = image[y0:y1, x0:x1]

            # Dados sobre o rosto
                # Reconhecimento Facial
            person = DeepFace.find(individual_face, db_path = "./db_rostos")
            per = 'N/A' if len(person[0]) == 0 else person[0].identity[0].split('/')[2].split('.')[0]
            per = per.lower()
                # Extração de características
            res = DeepFace.analyze(individual_face)[0]
            gender = res["dominant_gender"].lower()
            age = str(res["age"]).lower()
            emotion = res["dominant_emotion"].lower()
            race = res["dominant_race"].lower()

            # Inicio e Fim do rosto
            start_point = (x0, y0)
            end_point = (x1, y1)

            # Inicio e fim dos textos
            start_space = (x1, y0)
                # Final do texto seguirá o tamanho do maior texto escrito
            end_space = (x1+32+max(map(lambda x: cv2.getTextSize(x,cv2.FONT_HERSHEY_PLAIN,1, 1)[0][0], [race, emotion])), y0+105)

            # Cor e espessura do retrato
            color = (172, 52, 52)
            thickness = 2

            # Retrato na foto original e espaco para o texto
            image = cv2.rectangle(image, start_point, end_point, color, thickness)
            image = cv2.rectangle(image, start_space, end_space, color, -1) 

            # Resultados do rosto encontrado
            image = cv2.putText(image, f'P: {per}', (x1+5, y0+18), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1) 
            image = cv2.putText(image, f'G: {gender}', (x1+5, y0+38), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1) 
            image = cv2.putText(image, f'A: {age}', (x1+5, y0+58), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1) 
            image = cv2.putText(image, f'E: {emotion}', (x1+5, y0+78), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1) 
            image = cv2.putText(image, f'R: {race}', (x1+5, y0+98), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1) 
        except:
            pass
    return image

# Plataforma Web para uso do conhecimento

In [ ]:
import dash
import numpy as np
import dash_html_components as html
from dash_canvas import DashCanvas
from dash import dcc
from dash.dependencies import Input, Output, State
from dash_canvas.utils import array_to_data_url, image_string_to_PILImage
import warnings
warnings.filterwarnings("ignore")

app = dash.Dash(__name__)

canvas_width = 650

app.layout = html.Div([
    html.H2('Reconhecimento facial e análise de características', style={'text-align':'center'}),


    dcc.Upload(
        id='upload-image',
        children=html.Div([
            'Arraste e solte ou ',
            html.A('selecione uma imagem para analisar')
        ]),
        style={
            'width': '99%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        # Allow multiple files to be uploaded
        multiple=True
    ),
    html.Div(id='output-image-upload',style={'width': '100%', 'display': 'flex', 'align-items':'center', 'justify-content':'center'})
])


def parse_contents(contents, filename, date):
    img = image_string_to_PILImage(contents)
    img.save('img.jpg')
    img = cv2.imread("img.jpg")
    img = process_image(img)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pix = np.array(img)
    img_content = array_to_data_url(pix)
    return html.Div([DashCanvas(id='canvaas_image',
                                image_content=img_content,
                                lineWidth=5,
                                lineColor='red',
                                width=canvas_width)])


@app.callback(Output('output-image-upload', 'children'),
              Input('upload-image', 'contents'),
              State('upload-image', 'filename'),
              State('upload-image', 'last_modified'))
def update_output(list_of_contents, list_of_names, list_of_dates):
    if list_of_contents is not None:
        children = [
            parse_contents(c, n, d) for c, n, d in
            zip(list_of_contents, list_of_names, list_of_dates)]
        return children


if __name__ == '__main__':
    app.run_server(debug=True)


0: 640x480 5 Faces, 177.2ms
Speed: 36.5ms preprocess, 177.2ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
23-12-17 20:40:44 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-17 20:40:44 - There are 2 representations found in representations_vgg_face.pkl
23-12-17 20:40:44 - find function lasts 0.3456571102142334 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.25it/s]  


23-12-17 20:40:45 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-17 20:40:45 - There are 2 representations found in representations_vgg_face.pkl
23-12-17 20:40:45 - find function lasts 0.37296009063720703 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.41it/s]  


23-12-17 20:40:46 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-17 20:40:46 - There are 2 representations found in representations_vgg_face.pkl
23-12-17 20:40:47 - find function lasts 0.31015682220458984 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.41it/s]  


23-12-17 20:40:48 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-17 20:40:48 - There are 2 representations found in representations_vgg_face.pkl
23-12-17 20:40:48 - find function lasts 0.3055891990661621 seconds


Action: race: 100%|██████████| 4/4 [00:00<00:00,  4.27it/s]  


23-12-17 20:40:49 - ⚠️ Representations for images in ./db_rostos folder were previously stored in representations_vgg_face.pkl. If you added new instances after the creation, then please delete this file and call find function again. It will create it again.
23-12-17 20:40:49 - There are 2 representations found in representations_vgg_face.pkl
